<a href="https://colab.research.google.com/github/gt-cse-6040/topic_07_MT2_SP23_0510/blob/main/Topic%2007%20MT2%20SP23%20for%20Skills%20OH%20solution%20ex%200510.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Midterm 2, Spring 2023: Better Reads

_Version history:_
- 1.0.1 (Sun Apr 2): Corrected typo in Ex. 7 demo
- 1.0: Initial release

*All of the header information is important. Please read it.*

**Topics, number of exercises:** This problem builds on your knowledge of Numpy, pandas, database organization, graph abstractions, and basic Python (for interfacing with other Python libraries). It has **11** exercises, numbered 0 to **10**. There are **21** available points. However, to earn 100% the threshold is **12** points. (Therefore, once you hit **12** points, you can stop. There is no extra credit for exceeding this threshold.)

**Free points!** This exam includes one exercise, Exercise 3, whose points are "**free**." However, to get these points you need to read some text and _submit the notebook to the autograder at least once_.

**Exercise ordering:** Each exercise builds logically on previous exercises, but you may solve them in any order. Exercises are **not** necessarily ordered by difficulty, but higher point values usually imply more difficult tasks.

**Demo cells:** Code cells that start with the comment `### define demo inputs` will load results from prior exercises applied to the entire data set and use those to build demo inputs. These must be run for later demos to work properly but they do not affect the test cells. The data loaded by these cells may be large (at least in terms of human readability). You are free to inspect them, but we did not print them in the starter code.

**Debugging you code:** Right before each exercise test cell, there is a block of text explaining the variables available to you for debugging. You may use these to test your code and can print/display them as needed (careful when printing large objects, you may want to print the head or chunks of rows at a time).

**Exercise point breakdown:**

- Exercise 0: **2** points
- Exercise 1: **1** point
- Exercise 2: **3** points
- Exercise 3: **2** point **FREEBIE! Submit to record them**
- Exercise 4: **1** point
- Exercise 5: **2** points
- Exercise 6: **2** points
- Exercise 7: **1** point
- Exercise 8: **3** points
- Exercise 9: **2** points
- Exercise 10: **2** points

**Final reminders:**

- Submit after **every exercise**
- Review the generated grade report after you submit to see what errors were returned
- Stay calm, skip problems as needed, and take short breaks at your leisure

# Background: Better Reads #

[Goodreads](https://www.goodreads.com/) is a website devoted to curating user-generated book reviews. You'll do some elementary data-mining to uncover "communities" of users who like the same books. Such insights might help users find like-minded communities and generate better book recommendations.

**Overall workflow.** This notebook has six (6) parts with about 1-3 exercises each.
* **Part A:** Analyze user-book interactions [SQL, pandas]
* **Part B:** Power-law analysis [pandas, Numpy]
* **Part C:** Edge lists, NetworkX, and graph clusters [Python, graphs]
* **Part D:** Finding communities via graph clustering [SQL, pandas]
* **Part E:** Identifying "top reads" by community [pandas]
* **Part F:** Merging inventory metadata [pandas]

# Getting started (modules) #

Skim the code cell below and then run it. Take note of the standard preloaded modules, `numpy as np`, `pandas as pd`, and `sqlite3 as db`, any or all of which you may need to construct your solutions.

The other functions are used by our demo and testing code. You can ignore them unless an exercise asks you to do otherwise.

In [ ]:
# uncomment in Google Colab
# !python --version
!pip install dill
import dill as pickle

In [ ]:
### Global Imports
%load_ext autoreload
%autoreload 2

# Standard Python modules
import sys
import numpy as np
import pandas as pd
import sqlite3 as db
import networkx as nx

In [ ]:
### Global Imports
# Some functionality needed by the notebook and demo cells:
from pprint import pprint, pformat
import math

# === Iteration === #

def isiter(x):
    """
    Returns `True` if `x` is iterable.

    Uses the "duck typing" method described here:
    https://stackoverflow.com/questions/1952464/in-python-how-do-i-determine-if-an-object-is-iterable
    """
    try:
        iterator = iter(x)
    except TypeError:
        return False
    return True

def sample_iter(I, n=1, rng_or_seed=None, replace=False, safe=True):
    from pandas import DataFrame
    from numpy import ndarray, array

    rng = get_rng(rng_or_seed, ret_type=False)
    n_sample = min(n, len(I)) if safe else n
    sample_locs = rng.choice(range(len(I)), size=n_sample, replace=replace)
    if isinstance(I, DataFrame):
        sample = I.iloc[sample_locs]
    elif isinstance(I, ndarray) or isinstance(I, list):
        sample = I[sample_locs]
    elif isinstance(I, dict):
        sample_values = list(I.keys())[sample_locs]
        sample = {k: I[k] for k in sample_values}
    else:
        J = array(list(I))
        sample = type(I)(J[sample_locs])
    return sample

# === Messages === #

def status_msg(s, verbose=True, **kwargs):
    if verbose:
        print(s, **kwargs)

# === pandas ===

def subselect(df, col, values):
    """
    Subselects rows of a `DataFrame` where the column `col`
    contains any of the given `values`.

    If `values` is a non-iterable object _or_ a `str`,
    then this function treats it as a single value to
    find.
    """
    if not isinstance(values, str) and isiter(values):
        return df[df[col].isin(values)]
    return df[df[col] == values]

# === Input/output === #

# def text_to_file(s, basename, dirname='resource/asnlib/publicdata/', overwrite=True, verbose=True):
def text_to_file(s, basename, dirname='', overwrite=True, verbose=True):
    from os.path import isfile
    filename = f"{dirname}{basename}"
    status_msg(f"Writing string to '{filename}'...", verbose=verbose)
    if not overwrite and isfile(filename):
        status_msg(f"  ==> File exists already; skipping.", verbose=verbose)
    else:
        with open(filename, "wt") as fp:
            fp.write(s)
        status_msg(f"  ==> Done!", verbose=verbose)

# def load_text_from_file(basename, dirname='resource/asnlib/publicdata/', abort_on_error=False, verbose=False):
def load_text_from_file(basename, dirname='', abort_on_error=False, verbose=False):
    from os.path import isfile
    filename = f"{dirname}{basename}"
    status_msg(f"Loading string from '{filename}'...", verbose=verbose)
    if isfile(filename):
        try:
            with open(filename, "rt") as fp:
                s = fp.read()
            status_msg(f"  ==> Done!", verbose=verbose)
        except:
            if abort_on_error:
                raise
            else:
                status_msg(f"  ==> An error occurred.", verbose=verbose)
                s = ''
    return s

# def df_to_file(df, basename, dirname='resource/asnlib/publicdata/', overwrite=True, verbose=True):
def df_to_file(df, basename, dirname='', overwrite=True, verbose=True):
    from os.path import isfile
    from dill import dumps
    filename = f"{dirname}{basename}"
    if verbose:
        print(f"Writing `DataFrame` to '{filename}'...")
    if not overwrite and isfile(filename):
        print(f"  ==> File exists already; skipping.")
    else:
        with open(filename, "wb") as fp:
            fp.write(dumps(df))
        print(f"  ==> Done!")

# def load_df_from_file(basename, dirname='resource/asnlib/publicdata/', abort_on_error=False, verbose=False):
def load_df_from_file(basename, dirname='', abort_on_error=False, verbose=False):
    from os.path import isfile
    from dill import loads
    from pandas import DataFrame
    df = DataFrame()
    filename = f"{dirname}{basename}"
    status_msg(f"Loading `DataFrame` from '{filename}'...", verbose=verbose)
    if isfile(filename):
        try:
            with open(filename, "rb") as fp:
                df = loads(fp.read())
            status_msg(f"  ==> Done!", verbose=verbose)
        except:
            if abort_on_error:
                raise
            else:
                df = DataFrame()
                status_msg(f"  ==> An error occurred.", verbose=verbose)
    return df

# def obj_to_file(df, basename, dirname='resource/asnlib/publicdata/', overwrite=True, verbose=True):
def obj_to_file(df, basename, dirname='', overwrite=True, verbose=True):
    from os.path import isfile
    from dill import dumps
    filename = f"{dirname}{basename}"
    if verbose:
        print(f"Writing object (type `{type(df)}`) to '{filename}'...")
    if not overwrite and isfile(filename):
        print(f"  ==> File exists already; skipping.")
    else:
        with open(filename, "wb") as fp:
            fp.write(dumps(df))
        print(f"  ==> Done!")

# def load_obj_from_file(basename, dirname='resource/asnlib/publicdata/', abort_on_error=False, verbose=False):
def load_obj_from_file(basename, dirname='', abort_on_error=False, verbose=False):
    from os.path import isfile
    from dill import loads
    from pandas import DataFrame
    filename = f"{dirname}{basename}"
    status_msg(f"Loading object from '{filename}'...", verbose=verbose)
    if isfile(filename):
        try:
            with open(filename, "rb") as fp:
                df = loads(fp.read())
            status_msg(f"  ==> Done! Type: `{type(df)}`", verbose=verbose)
        except:
            if abort_on_error:
                raise
            else:
                df = DataFrame()
                status_msg(f"  ==> An error occurred.", verbose=verbose)
    else:
        df = None
    return df

# def load_table_from_db(table_name, basename, dirname="resource/asnlib/publicdata/", verbose=False):
def load_table_from_db(table_name, basename, dirname="", verbose=False):
    from sqlite3 import connect
    from pandas import read_sql
    filename = f"{dirname}{basename}"
    if verbose:
        print(f"Retrieving table `{table_name}` from SQLite3 DB `{filename}`...")
    conn = connect(f"file:{filename}?mode=ro", uri=True)
    df = read_sql(f"SELECT * FROM {table_name}", conn)
    conn.close()
    if verbose:
        print(f"... done! Found {len(df)} rows.")
    return df

# ==== RNGs ==== #

# https://stackoverflow.com/questions/279561/what-is-the-python-equivalent-of-static-variables-inside-a-function
def static_vars(**kwargs):
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

@static_vars(DEFAULT_RNG=None)
def get_rng(rng_or_seed, ret_type=True):
    """
    Returns a valid pseudorandom-number generator (RNG) object
    based on how `rng_or_seed` is set:

    - An integer: Creates a new RNG with the integer as a seed
    - An existing RNG object: Returns the same object
    - `None`: Returns a global "default" RNG, which is created
      once at module initialization time.

    If `ret_type` is set, this function also returns a descriptive
    string saying which of the above cases applies. (The intent of
    this string is for use in printing as part of debugging output.)
    """
    # Initialize static variable, DEFAULT_RNG
    from numpy.random import default_rng
    if get_rng.DEFAULT_RNG is None:
        get_rng.DEFAULT_RNG = default_rng(1_234_567_890)

    if isinstance(rng_or_seed, int):
        rng = default_rng(rng_or_seed)
        rng_type = f'`default_rng({rng_or_seed})`'
    elif rng_or_seed is None:
        rng = get_rng.DEFAULT_RNG
        rng_type = f'`DEFAULT_RNG` [{rng}]'
    else:
        rng = rng_or_seed # had better be a RNG
        rng_type = f'User-supplied [{rng}]'

    return (rng, rng_type) if ret_type else rng

# ==== Plotting ==== #
def plot_series_loglog(series, ax=None, figsize=(8, 8/16*9), **kwargs):
    from matplotlib.pyplot import figure, gca
    if ax is None:
        fig = figure(figsize=figsize)
        ax = gca()
    x = series.index
    y = series.values
    ax.loglog(x, y, '.', **kwargs)
    return ax

def display_image_from_file(filename, verbose=False):
    from IPython.display import Image
    if verbose:
        print(f"Loading image, `{filename}` ...")
    display(Image(filename))
    if verbose:
        print(f"... done! (Did it appear?)")

# ==== Graph / NetworkX interfacing ===== #

def to_nx(edge_list):
    from networkx import DiGraph
    G = DiGraph()
    G.add_weighted_edges_from(edge_list)
    return G

def graph_to_matrix(G):
    try:
        from networkx import to_scipy_sparse_array # Works in 3.0
        return to_scipy_sparse_array(G)
    except:
        pass

    try:
        from networkx import to_scipy_sparse_matrix # Works in 2.5
        return to_scipy_sparse_matrix(G)
    except:
        raise

def graph_spy(G, style='matrix', ax=None, figsize=(6.5, 6.5), **kwargs):
    from matplotlib.pyplot import figure, gca
    from networkx import spring_layout, draw_networkx_nodes, draw_networkx_edges, draw_networkx_labels

    if ax is None:
        fig = figure(figsize=figsize)
        ax = gca()

    if style == 'matrix':
        A = graph_to_matrix(G)
        ax.spy(A, **kwargs)
    else:
        pos = spring_layout(G, seed=7)

        # nodes
        draw_networkx_nodes(G, pos) #, node_size=700)

        # edges
        elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] >= 0.1]
        esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] < 0.1]
        draw_networkx_edges(G, pos, edgelist=elarge, width=2)
        draw_networkx_edges(G, pos, edgelist=esmall, width=0.5, alpha=0.5)

        # node labels
        draw_networkx_labels(G, pos, font_size=10, font_family="sans-serif")
        # edge weight labels
#        edge_labels = nx.get_edge_attributes(G, "weight")
#        nx.draw_networkx_edge_labels(G, pos, edge_labels)
    return ax

def detect_communities(G, seed=1_234):
    from networkx.algorithms.community import louvain_communities
    return louvain_communities(G, seed=seed)

def random_clusters(nc, nvc, p_intra=0.5, p_inter=0.1, rng_or_seed=None, verbose=False):
    rng = get_rng(rng_or_seed, ret_type=False)
    n = nc * nvc
    mv_intra = int(p_intra*nvc) + 1 # no. of intra-cluster edges per vertex
    mv_inter = int(p_inter*n)       # no. of inter-cluster edges per vertex

    if verbose:
        print('Constructing a vertex-clustered graph with these properties:')
        print(f'- Number of clusters: nc={nc}')
        print(f'- Vertices per cluster: nvc={nvc}')
        print(f'- Number of intra-cluster edges per vertex: {mv_intra} (p_intra={p_intra})')
        print(f'- Number of inter-cluster edges per vertex: {mv_inter} (p_inter={p_inter})')
        print(f'- RNG: {rng}')

    V = set(range(n)) # `n` vertices
    E = []
    for c in range(nc):
        V_c = set(range(c*nvc, (c+1)*nvc))
        for v in V_c:
            # Add intra-cluster edges for `v`
            N_v = sample_iter(V_c - {v}, n=mv_intra, rng_or_seed=rng)
            W_v = rng.random(size=len(N_v))
            E += [(v, u, w) for u, w in zip(N_v, W_v)]

            # Add inter-cluster edges for `v`
            X_v = sample_iter(V - V_c, n=mv_inter, rng_or_seed=rng)
            W_v = rng.random(size=len(X_v)) / 10.0 # make these edges weaker, too
            E += [(v, u, w) for u, w in zip(X_v, W_v)]
    return E

In [ ]:
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/demo-comm-vecs.png
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/demo_ex0.db
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/ex0.txt
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/ex10-final.df
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/ex5.df
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/goodreads.db
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/tc_0
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/tc_10
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/tc_5

!mkdir tester_fw
%cd tester_fw

!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/tester_fw/__init__.py
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/tester_fw/test_utils.py
!wget https://raw.githubusercontent.com/gt-cse-6040/topic_07_MT2_SP23_0510/main/tester_fw/testers.py

%cd ..

In case it's helpful, here are the versions of Python and standard modules you are using:

In [ ]:
print("* Python version: {}".format(sys.version.replace('\n', ' ')))
print(f"* Numpy version: {np.__version__}")
print(f"* pandas version: {pd.__version__}")
print(f"* sqlite3 version: {db.version}")

## pandas versus SQL ##

The actual Goodreads data is provided via a SQLite3 database. However, only some exercises _require_ SQL; most exercises were designed with pandas in mind.

Nevertheless, even some of the pandas exercises can be solved using SQL. The cell below defines the function, `dfs_to_conn`, which can be used to create in-memory database connections. If you pass in a dictionary mapping table names to pandas `DataFrame` objects, then `dfs_to_conn` will return a `sqlite3` connection with all of the data in the `DataFrame` objects available under the names given as keys. You are also free to write to the in-memory database by creating tables, inserting, deleting, updating records, etc. Anything that SQLite3 allows should work.

**Example:**

```python
    my_df = pd.DataFrame({'A':[1,2,3], 'B': [4,5,6], 'C':['x', 'y', 'z']})
    print(my_df)
    #    A  B  C
    # 0  1  4  x
    # 1  2  5  y
    # 2  3  6  z
    conn = dfs_to_conn({'my_table': my_df})
    cur = conn.cursor()
    cur.execute('select A, B, C from my_table')
    result = cur.fetchall()
    conn.close()
    print(result) # list of tuples, each tuple is a row
    #[(1, 4, 'x'), (2, 5, 'y'), (3, 6, 'z')]
```

In [ ]:
def dfs_to_conn(conn_dfs, index=False):
    import sqlite3
    conn = sqlite3.connect(':memory:')
    for table_name, df in conn_dfs.items():
        df.to_sql(table_name, conn, if_exists='replace', index=index)
    return conn

# Goodreads Data (`grdbconn`) #

Some of the Goodreads data is stored in a SQLite3 database. The code cell below opens a read-only connection to it named **`grdbconn`**.

For now, don't worry about what's there. We will explain any tables you need in the exercises that use them.

In [ ]:
# Goodreads database connection:
grdbconn = db.connect('file:goodreads.db?mode=ro', uri=True)

# Part A: Analyzing user-book interactions #

> Includes Exercise 0 (2 points) and Exercise 1 (1 point).

The Goodreads dataset includes **user-book interactions.** An "user-book interaction" means the user "did something" with the book on the Goodreads website:

- _Viewed_: The user looked at a book description and saved it to their personal library.
- _Read_: The user marked the book as "read."
- _Rated_: The user gave the book a rating, from 1 to 5 stars.
- _Reviewed_: The user wrote a public review of the book on the website.

These interactions are recorded in a SQL table called `Interactions`. Let's have a quick look for one of the users whose integer ID is `840218`:

In [ ]:
pd.read_sql(r"SELECT * FROM Interactions WHERE user_id=830690", grdbconn)

Each row shows how this user interacted with some book. This user interacted with five books. However, they saved books `15396` (row 0) and `39407` (row 4) but did nothing else with them—that is, they did not read them, rate them, or review them.

They did rate books `19990`, giving it `5` stars, as well as `19989`, giving it `3` stars. They also read `19988`, but they did not rate it. They did not review any book (`is_reviewed=0`). Had they done so, `is_reviewed` would be `1`. All values are integers.

## **Ex. 0 (2 pts)**: `summarize_interactions_str` ##

You are asked to write a summary report of the overall interactions. Complete the function
```python
def summarize_interactions_str(conn):
    ...
```
so that it does the following.

**Inputs:** The input is a SQLite3 database connection containing a table named `Interactions` with the fields `user_id`, `book_id`, `is_read`, `rating`, and `is_reviewed`, all containing integer values.

**Your task:** Calculate the following:

- The total number of interactions
- The number of _unique_ user IDs
- The number of _unique_ book IDs
- The number of interactions where the user ...
  - read the book, i.e., where `is_read` equals `1`;
  - rated the book, i.e., where `rating` is _greater than_ `0`;
  - reviewed the book, i.e., where `is_review` equals `1`.

**Output:** Generate and **return a string** that reports these results. The string should be formatted as follows:
```
There are 370,818 interactions.
- Unique users: 2,000
- Unique books: 138,633
- Number of reads: 208,701 (56.3% of all interactions)
- Number of ratings: 194,243 (52.4%)
- Number of reviews: 23,720 (6.4%)
```
In particular:
- Commas should be used every three digits to make the numbers more readable.
- The percentages should be reported to one digit after the decimal place (even if that digit is `0`, e.g., `37.0%`).
- Newlines should appear between lines as shown in the example. However, there should be _no_ leading or trailing whitespace.

**Additional notes and hints:**
1. The function predefines a string template for the report. It's probably easiest to **modify** this template to achieve the desired result.
2. You may assume that there are no duplicate (`user_id`, `book_id`) pairs.
3. Recall that Python f-strings can help format numbers. See the demo cell below.

In [ ]:
### Demo: Recall Python's f-strings

print(f"`pi` to 2 decimal digits: `{3.14159265358979:0.2f}`")
print(f"Behold: `{1234567890:,}` -- neat!")

In [ ]:
### Define demo inputs

demo_conn_ex0 = db.connect(f'file:demo_ex0.db?mode=ro', uri=True)
print("First five rows of the demo database:")
pd.read_sql(r"SELECT * FROM Interactions LIMIT 5", demo_conn_ex0)

The demo included in the solution cell below should display the following output:
```
There are 12,345 interactions.
- Unique users: 1,766
- Unique books: 9,348
- Number of reads: 6,844 (55.4% of all interactions)
- Number of ratings: 6,389 (51.8%)
- Number of reviews: 744 (6.0%)
```

In [ ]:
### Exercise 0 solution ###
def summarize_interactions_str(conn):
    # Use or adapt this template as you see fit:
    template = """There are {} interactions.
- Unique users: {}
- Unique books: {}
- Number of reads: {} ({}% of all interactions)
- Number of ratings: {} ({}%)
- Number of reviews: {} ({}%)"""

    ### BEGIN SOLUTION
    df = pd.read_sql("SELECT * FROM Interactions", conn)
    n_users = len(df['user_id'].unique())
    n_books = len(df['book_id'].unique())
    n_reads = df['is_read'].sum()
    n_rated = (df['rating'] > 0).sum()
    print(df['rating'] > 0)
    n_rev = df['is_reviewed'].sum()
    template = f"""There are {len(df):,} interactions.
- Unique users: {n_users:,}
- Unique books: {n_books:,}
- Number of reads: {n_reads:,} ({n_reads/len(df)*100:.1f}% of all interactions)
- Number of ratings: {n_rated:,} ({n_rated/len(df)*100:.1f}%)
- Number of reviews: {n_rev:,} ({n_rev/len(df)*100:.1f}%)"""
    return template
    ### END SOLUTION

### demo function call
print(summarize_interactions_str(demo_conn_ex0))

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 0. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex0
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_0',
    'func': summarize_interactions_str, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'conn':{
            'dtype': 'db', # data type of param.
            'check_modified': False,
        }
    },
    'outputs':{
        'output_0': {
            'index': 0,
            'dtype': 'str',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': True, # Ignored if dtype is not df
            'check_row_order': True, # Ignored if dtype is not df
            'check_column_type': True, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(10):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

**RUN ME:** A correct implementation of `summarize_interactions_str`, when run on the full Goodreads dataset, would produce the following report:

In [ ]:
print(f"\n=== Report on the full dataset ===\n\n{load_text_from_file('ex0.txt')}")

## **Ex. 5 (2 pts)**: `connect_users` ##

Given the analysis sample from Exercise 4, let's "connect" users.

Let's say that two users `a` and `b` are **connected** if they both gave ratings of 4 or higher to the same book. The number of unique books they both rated this way is a measure of how strong their connection is.

Complete the following function to help identify these connections.
```python
def connect_users(ubdf, threshold):
    ...
```

**Inputs:**
- `ubdf`: A "user-book" dataframe having these two columns: `user_id` and `book_id`. Each row indicates that a given user gave a given book a rating of 4 or higher.
- `threshold`: An integer threshold on connection strength.

**Your tasks:** Determine which pairs of users are connected. Count how many books connect them. Drop self-pairs (`user_id_x == user_id_y`), as well as any pairs with fewer than `threshold` connections.

**Outputs:** Return a **new** `DataFrame` with three columns:
1. `user_id_x`: A user ID
2. `user_id_y`: Another user ID
3. `count`: The number of books they both rated in common. Recall that this value should be `>= threshold`.

**Additional notes and hints.**
1. Omit self-pairs, that is, cases where `user_id_x` == `user_id_y`.
1. Return pairs **symmetrically**. That is, if the pair of users (`a`, `b`) have a count `k` at or above the threshold, then **both** (`a`, `b`, `k`) and (`b`, `a`, `k`) should be rows in the output table.
1. If no connections meet the threshold, you should return an empty `DataFrame` _with_ the specified columns.
1. You may assume there are no duplicate rows.

> _Aside:_ For really huge datasets (not what is included in this exam), dropping users with fewer than `threshold` ratings _before_ looking for pairs might be a bit faster.

**Example:** Suppose the inputs are the `DataFrame` shown below with a target connection threshold of `2`:

In [ ]:
### Define demo inputs ###

demo_ubdf_ex5 = load_df_from_file("demo_ex5.df").sort_values(['book_id', 'user_id']).reset_index(drop=True)
demo_threshold_ex5 = 2

display(demo_ubdf_ex5)

For this input, `connect_users` should produce:

|   user_id_x |   user_id_y |   count |
|------------:|------------:|--------:|
|           0 |           2 |       2 |
|           0 |           3 |       2 |
|           2 |           0 |       2 |
|           3 |           0 |       2 |

Users `0` and `2` both rated books `7` and `19`, so they meet the threshold of having reviewed 2 books in common. User `1` did not review any books in common with any other user, and so they do not appear in any pair of the output.

In [ ]:
### Exercise 5 solution
def connect_users(ubdf, threshold):
    ### BEGIN SOLUTION
    uudf = ubdf.merge(ubdf, on='book_id') \
               .groupby(['user_id_x', 'user_id_y']) \
               .size() \
               .reset_index() \
               .rename(columns={0: 'count'})
    uudf = uudf[uudf['user_id_x'] != uudf['user_id_y']]
    uudf = uudf[uudf['count'] >= threshold]
    uudf = uudf.reset_index(drop=True)
    return uudf
    ### END SOLUTION

### demo function call ###
connect_users(demo_ubdf_ex5, demo_threshold_ex5)

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 5. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex5
from tester_fw.testers import Tester

conf = {
    'case_file':'tc_5',
    'func': connect_users, # replace this with the function defined above
    'inputs':{ # input config dict. keys are parameter names
        'ubdf': {
            'dtype': 'df', # data type of param.
            'check_modified': True
        },
        'threshold': {
            'dtype': 'int',
            'check_modified': False
        }
    },
    'outputs':{
        'output_0':{
            'index': 0,
            'dtype': 'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

**RUN ME:** From a correct implementation of `connect_users`, one way we can "draw" the connectivity is to form a sparse matrix where nonzeros represent connections. Here is a picture of this matrix for the full dataset, using a threshold of 2:

In [ ]:
uudf = load_df_from_file('ex5.df') # user-user table

print("A sample of connections:")
display(uudf.head())

if False: # Disabled due to NetworkX version incompatibility issue (fix pending)
    uudf_G = cse6040.utils.to_nx(uudf.to_records(index=False))
    ax_ex5 = cse6040.utils.graph_spy(uudf_G, markersize=0.01)
    ax_ex5.set_title('Spy plot: user-user interactions')
    ax_ex5.set_xlabel('user id')
    ax_ex5.set_ylabel('user id', rotation=0, horizontalalignment='right');
else:
#     cse6040.utils.display_image_from_file('demo-user-user-spy.png')
    pass

> **Aside (skip if pressed for time):** The "grid-like" pattern you might see suggests that there are groups or clusters of interconnected users in the data. Our next task will try to identify them.

# Part F (final part!): Merging inventory metadata #

> Includes Exercises 9 and 10 (2 points each).


To interpret the communities, we need to bring in some book-inventory metadata, like book titles and genres. Once we've done so, will the communities make sense?

## Genre vectors ##

The original dataset includes information on _genres_ for each book:

In [ ]:
genres = pd.read_sql("SELECT * FROM Genres", grdbconn)
genres

It's a bit messy, however: the genre information is stored as a JSON-formatted Python string encoding a **genre vector**:

In [ ]:
# Inspect the very first genre entry:
print(f"* Type: `{type(genres['genres'].iloc[0])}`")
print(f"* Value: '{genres['genres'].iloc[0]}'")

This genre vector says that this particular book mixes three genres: `fiction`, `romance`, and `"mystery, thriller, crime"` (considered a single genre). Each value measures the relevance of that genre to the book.

> Roughly speaking, let's interpret `555` as meaning this book is 555 / (555+23+10) ~ 94.3% "fiction" and 23 / (555+23+10) ~ 3.9% "romance."

The database stores these as genre vectors as strings. However, we can easily convert them to Python dictionaries using the following helper function, `from_json_str`:

In [ ]:
def from_json_str(s):
    """Parses the JSON string `s` and returns a Python object."""
    from json import loads
    return loads(s)


# Demo #

print("iloc 0:", from_json_str(genres['genres'].iloc[0]))
print("iloc 1:", from_json_str(genres['genres'].iloc[1]))

We will treat these as (mathematical) vectors that we can "add." Here is a simple function to compute the sum of two genre vectors:

In [ ]:
def add_genre_vecs(x, y):
    """Returns the sum of two genre vectors."""
    from collections import defaultdict
    z = defaultdict(int)
    for k, v in x.items():
        z[k] += v
    for k, v in y.items():
        z[k] += v
    return dict(z) # Converts into a regular Python dict

# Demo: start with two genre vectors, converted to `dict`:
demo_genre_vec_a = from_json_str(genres['genres'].iloc[0])
demo_genre_vec_b = from_json_str(genres['genres'].iloc[1])

# Add them:
add_genre_vecs(demo_genre_vec_a, demo_genre_vec_b)

## **Ex. 10 (2 pts)**: `combine_all_data` ##

The final step in our analysis is to combine several pieces of information into a final `DataFrame`. In particular, we'd like to take the "top reads" results from Exercise 8 and add in (a) book titles and (b) book genres. Complete the function so that it carries out this task.

```python
def combine_all_data(topdf, book2inv, invdf, genresdf):
    ...
```

**Inputs:** The inputs consist of **four** `DataFrame` objects.
- **`topdf`**: A dataframe of the top reads by community (e.g., from Ex. 8). Its columns are:
  * `'comm_id'`: An integer community ID
  * `'book_id'`: An integer book ID
  * `'comm_size'`: The number of users in the community
  * `'percent'`: The percentage of community users that read the given book
- **`book2inv`**: A dataframe to convert book IDs into _"inventory IDs."_ It has two columns:
  * `'book_id'`: An integer book ID
  * `'inv_id'`: An inventory ID, which can be used to link the book to its title and genre
- **`invdf`**: An inventory of books. Its columns include:
  * `'inv_id'`: An integer inventory ID
  * `'title'`: The book's title, a string
  * `'description'`: A brief description of the book
- **`genres`**: Genre vectors, encoded as JSON strings. Its columns are:
  * `'inv_id'`: The integer inventory ID
  * `'genres'`: The genre vector (as a JSON string)

**Your task:** Merge all of this data into a single `DataFrame`. You should perform a series of left-merges (pandas equivalent of left-joins), starting with `topdf`, using either `book_id` or `inv_id` to link the dataframes. By doing left-joins, you will preserve all the rows of `topdf`.

**Outputs:** Your function should return the `DataFrame`. It will have only the columns listed above: `'comm_id'`, `'book_id'`, `'comm_size'`, `'percent'`, `'inv_id'`, `'title'`, `'description'`, `'genres'`.

**Additional notes:** You do not need to convert any of the fields, you just need to arrange the merges correctly.

**Example:** The following cell loads some demo inputs that you can use for testing and debugging. (Because there are several of these, we have refrained from printing them. However, you can use the next cell to write code to explore them.)

In [ ]:
### Define demo inputs ###

demo_topdf_ex10 = load_df_from_file("demo_ex10-topdf.df")
demo_book2inv_ex10 = load_df_from_file("demo_ex10-book2inv.df")
demo_invdf_ex10 = load_df_from_file("demo_ex10-invdf.df")
demo_genresdf_ex10 = load_df_from_file("demo_ex10-genresdf.df")

In [ ]:
# Use this cell to `display`, `print`, or otherwise explore those demo inputs

A correctly functioning `combine_all_data` will produce the following output on the demo inputs:

|   comm_id |   book_id |   comm_size |   percent |   inv_id | title                                                       | description   | genres                                                                                                                     |
|----------:|----------:|------------:|----------:|---------:|:------------------------------------------------------------|:--------------|:---------------------------------------------------------------------------------------------------------------------------|
|         0 |       821 |         868 |   22.5806 |     5470 | 1984                                                        | The year 1... | {"fiction": 25686, "fantasy, paranormal": 1776, "young-adult": 233}                                                        |
|         0 |       943 |         868 |   21.4286 |        3 | Harry Potter and the Sorcerer's Stone (Harry Potter, #1)    | Harry Pott... | {"fantasy, paranormal": 54156, "young-adult": 17058, "fiction": 15016, "children": 11213, "mystery, thriller, crime": 668} |
|         2 |     49734 |          36 |   22.2222 |  6604887 | أنت لي                                                      |               | {"romance": 31, "fiction": 9}                                                                                              |
|         2 |     23164 |          36 |   19.4444 |  7704143 | تراب الماس                                                  | "llmr@ lth... | {"fiction": 27, "mystery, thriller, crime": 32}                                                                            |
|         3 |       943 |         340 |   77.0588 |        3 | Harry Potter and the Sorcerer's Stone (Harry Potter, #1)    | Harry Pott... | {"fantasy, paranormal": 54156, "young-adult": 17058, "fiction": 15016, "children": 11213, "mystery, thriller, crime": 668} |
|         3 |       941 |         340 |   74.1176 |        5 | Harry Potter and the Prisoner of Azkaban (Harry Potter, #3) | Harry Pott... | {"fiction": 12103, "children": 8558, "fantasy, paranormal": 4639, "young-adult": 1513, "mystery, thriller, crime": 537}    |
|         4 |    139433 |           6 |   50      |   148849 | شازده کوچولو                                                | shzdh khwc... | {"fiction": 5481, "fantasy, paranormal": 3847, "children": 8886, "young-adult": 1127}                                      |

In [ ]:
### Exercise 10 solution
def combine_all_data(topdf, book2inv, invdf, genresdf):
    ### BEGIN SOLUTION
    return topdf[['comm_id', 'book_id', 'comm_size', 'percent']] \
            .merge(book2inv[['book_id', 'inv_id']], on='book_id', how='left') \
            .merge(invdf[['inv_id', 'title', 'description']], on='inv_id', how='left') \
            .merge(genresdf, on='inv_id', how='left') \
            .sort_values(['comm_id', 'percent'], ascending=[True, False])
    ### END SOLUTION

### demo function call ###
# combine_all_data(demo_topdf_ex10, demo_book2inv_ex10, demo_invdf_ex10, demo_genresdf_ex10)

<!-- Test Cell Boilerplate -->
The cell below will test your solution for Exercise 10. The testing variables will be available for debugging under the following names in a dictionary format.
- `input_vars` - Input variables for your solution.
- `original_input_vars` - Copy of input variables from prior to running your solution. These _should_ be the same as `input_vars` - otherwise the inputs were modified by your solution.
- `returned_output_vars` - Outputs returned by your solution.
- `true_output_vars` - The expected output. This _should_ "match" `returned_output_vars` based on the question requirements - otherwise, your solution is not returning the correct output.

In [ ]:
### test_cell_ex10
from tester_fw.testers import Tester

conf = {
    'case_file': 'tc_10',
    'func': combine_all_data, # replace this with the function defined above
    'inputs': { # input config dict. keys are parameter names
        'topdf': {'dtype': 'df', 'check_modified': True},
        'book2inv': {'dtype': 'df', 'check_modified': True},
        'invdf': {'dtype': 'df', 'check_modified': True},
        'genresdf': {'dtype': 'df', 'check_modified': True},
    },
    'outputs': {
        'output_0': {
            'index': 0,
            'dtype': 'df',
            'check_dtype': True,
            'check_col_dtypes': True, # Ignored if dtype is not df
            'check_col_order': False, # Ignored if dtype is not df
            'check_row_order': False, # Ignored if dtype is not df
            'float_tolerance': 10 ** (-6)
        }
    }
}
tester = Tester(conf, key=b'jpS7W-CpqAQfuITMEQZL-yVXfhIaCkSaei-emnyRtrI=', path='')
for _ in range(70):
    try:
        tester.run_test()
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
    except:
        (input_vars, original_input_vars, returned_output_vars, true_output_vars) = tester.get_test_vars()
        raise

print('Passed! Please submit.')

**RUN ME:** If `combine_all_data` is working and applied to the full Goodreads dataset, here are the results:

In [ ]:
ex10_final = load_df_from_file('ex10-final.df')
ex10_final_groups = ex10_final.groupby('comm_id')
for comm_id in ex10_final_groups.groups.keys():
    display(ex10_final_groups.get_group(comm_id))

> Scan the titles, descriptions, and genres. Do the community labels appear to identify distinct communities?

# Fin! #

If you have made it this far, that's it — congratulations on completing the exam. **Don't forget to submit!**

In [ ]:
# Close database connection
try:
    grdbconn.close()
except:
    print("Goodreads database-connection may already be closed.")

**Postscript.** Had you gotten everything right, then we could have performed one final analysis on the previous result.

Suppose you calculate the normalized genre vectors for each community, and then plot the components for each community as shown below.

![Genre vectors uncovered](https://github.com/gt-cse-6040/topic_07_MT2_SP23_0510/blob/main/demo-comm-vecs.png?raw=1)

Darker bars correspond to more highly weighted components. You can see that the community genre-vectors are distinct from one another, albeit with some (expected) overlaps. Thus, there is, arguably, at least some additional evidence to suspect this initial grouping may be a meaningful one for helping users find other users and appropriate book recommendations. The analysis in this notebook operated on just a small fraction of the complete dataset, and it is possible that with more data more distinct communities could emerge.

**Want to explore this dataset on your own?** Refer to the [Goodreads Dataset](https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home). It was originally collected in 2017 by researchers at the University of California, San Diego. It is quite extensive, and what we did in this exam barely scratches the surface of what is possible!